In [19]:
import numpy as np    
import pandas as pd  
from apyori import apriori 
ratings = pd.read_csv('C:\\Users\\User\\Desktop\\Workspace\\ML\\recommendation_engine\\input\\ml_small\\ratings.csv') 
movies = pd.read_csv('C:\\Users\\User\\Desktop\\Workspace\\ML\\recommendation_engine\\input\\ml_small\\movies.csv') 

movie_ratings = pd.merge(ratings[['userId','movieId','rating']],movies[['movieId','title']],on='movieId')


ratings_count = movie_ratings.groupby(by="title")['rating'].count().reset_index().rename(columns={'rating':'totalRatings'})[['title','totalRatings']]
all_ratings = pd.merge(movie_ratings,ratings_count,on='title',how='left')

votes_count_threshold = 60
ratings_top = all_ratings.query('totalRatings > @votes_count_threshold')
print('running for : ', ratings_top.shape)
#ratings_top.shape

if not ratings_top[ratings_top.duplicated(['userId','title'])].empty:
    ratings_top = ratings_top.drop_duplicates(['userId','title'])
    
#ratings_top.shape  - 66658,5
record_set = ratings_top.pivot(index='title',columns='userId',values='rating').fillna(0)
record_set =record_set.fillna(0)

def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

records_T = record_set.T.applymap(encode_units)   
records_T = records_T.fillna(0)
#records.head()

row,col = records_T.shape

running for :  (34660, 5)


In [20]:
records  = []
for i in range(0, row):  
    records.append([str(records_T.columns.values[j]) for j in range(0, col) if records_T.values[i,j] > 0])

association_rules = apriori(records,min_support = 0.2,min_confidence=0.8,min_lift = 2,min_length=2) 
association_results = list(association_rules)

print(len(association_results))
for i in association_results:
    print(i)    

56
RelationRecord(items=frozenset({'Beauty and the Beast (1991)', 'Aladdin (1992)'}), support=0.20398009950248755, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Beauty and the Beast (1991)'}), items_add=frozenset({'Aladdin (1992)'}), confidence=0.8424657534246575, lift=2.7912464248080684)])
RelationRecord(items=frozenset({'Godfather: Part II, The (1974)', 'Godfather, The (1972)'}), support=0.20729684908789386, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Godfather: Part II, The (1974)'}), items_add=frozenset({'Godfather, The (1972)'}), confidence=0.9689922480620156, lift=3.043241279069768)])
RelationRecord(items=frozenset({'Independence Day (a.k.a. ID4) (1996)', 'Mission: Impossible (1996)'}), support=0.21393034825870647, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Mission: Impossible (1996)'}), items_add=frozenset({'Independence Day (a.k.a. ID4) (1996)'}), confidence=0.8012422360248447, lift=2.415745341614907)])
RelationRecord(items=frozens

In [45]:
input_id = 480
for item in association_results:
    pair = item[0]
    items = [x for x in pair]
    if  str(movies.loc[movies['movieId']==input_id]['title'].values[0]) in items[0] :
            print('Rule: ' + items[0] + "->" + items[1])
            print('Support: ' + str(item[1]))
            print('Confidence: ' + str(item[2][0][2]))
            print('Lift: ' + str(item[2][0][3]))            
            
        
    

Rule: Jurassic Park (1993)->Mask, The (1994)
Support: 0.20729684908789386
Confidence: 0.8012820512820513
Lift: 2.038704965920156
Rule: Jurassic Park (1993)->Speed (1994)
Support: 0.22553897180762852
Confidence: 0.8
Lift: 2.0354430379746837
Rule: Jurassic Park (1993)->Forrest Gump (1994)
Support: 0.20563847429519072
Confidence: 0.8493150684931506
Lift: 2.1609155540142186
Rule: Jurassic Park (1993)->Fugitive, The (1993)
Support: 0.20563847429519072
Confidence: 0.8378378378378379
Lift: 2.1317139924734865
Rule: Jurassic Park (1993)->Independence Day (a.k.a. ID4) (1996)
Support: 0.20398009950248755
Confidence: 0.8424657534246575
Lift: 2.1434888156753944
Rule: Jurassic Park (1993)->Speed (1994)
Support: 0.20066334991708126
Confidence: 0.8402777777777778
Lift: 2.13792194092827
Rule: Jurassic Park (1993)->Terminator 2: Judgment Day (1991)
Support: 0.23051409618573798
Confidence: 0.8224852071005917
Lift: 2.0926522357875816
Rule: Jurassic Park (1993)->Pulp Fiction (1994)
Support: 0.2205638474295

In [44]:
movies.loc[movies['title']=='Jurassic Park (1993)']

,movieId,title,genres
418,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
